In [1]:
!nvidia-smi

Fri Dec 23 23:24:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 45%   30C    P8    N/A /  75W |    469MiB /  4096MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow as tf

gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

# Obraz - transfer learning - praca domowa
Ostatnia aktualizacja: 2022.12.10

Z dokumentacji [Keras](https://keras.io/guides/transfer_learning/): *Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.*

Głównym celem pracy domowej jest stworzenie klasyfikatora, który będzie odróżniał zdjęcia obiektów w wybranym przez nas zbiorze. Użyjemy do tego jednego z [gotowych modeli Keras](https://keras.io/api/applications/), wytrenowanego wcześniej na zbiorze Imagenet. 

Przydatne źródła:
- [transfer learning vgg16 + tf_flowers](https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4)
- [Keras - transfer learning](https://keras.io/guides/transfer_learning/)

### Wybrany zbiór danych:

- [znaki drogowe](https://www.kaggle.com/datasets/valentynsichkar/traffic-signs-1-million-images-for-classification). 


In [2]:
import h5py
filename = "dataset_ts_original.hdf5"

with h5py.File(filename, "r") as f:
    
    x_test = f['x_test'][()].astype('float32')
    x_train = f['x_train'][()].astype('float32')
    y_test = f['y_test'][()].astype('float32')
    y_train = f['y_train'][()].astype('float32')
    x_validation = f['x_validation'][()].astype('float32')
    y_validation = f['y_validation'][()].astype('float32')

### Zadanie 1: Wytrenuj model na swoich danych ###
**(Zadanie na ocenę 3)**

Wytrenuj wybrany model na swoich danych. Omów eksperyment i wyniki (100 słów). 

In [15]:
assert x_test.shape[1:] == x_train.shape[1:] == x_validation.shape[1:]
print(x_test.shape)
print(x_train.shape)
print(x_validation.shape)

(3111, 48, 48, 3)
(36288, 48, 48, 3)
(12440, 48, 48, 3)


In [6]:
# x_validation_resized = tf.image.resize(x_validation[:300], (150, 150)).numpy()
# x_test_resized = tf.image.resize(x_test[:50], (150, 150)).numpy()
# x_train_resized = tf.image.resize(x_train[:1500], (150, 150)).numpy()

In [2]:
x_train[0].shape

(48, 48, 3)

### Wczytaj wytrenowany model do klasyfikacji obrazu. ###
Może to być jeden z gotowych [modeli dostępnych w Keras](https://keras.io/api/applications/). Wczytujemy go z wytrenowanymi już wcześniej wagami na Imagenecie (weights='imagenet'). 

Model możemy wczytać bez ostatnich warstw (include_top=False) i dodać je potem ręcznie, dostosowane do liczby klas w naszym zbiorze. Imagenet ma 1000 klas, my prawdopodobnie będziemy mieć ich mniej. 

Pamiętaj, żeby wyłączyć lub ograniczyć trening części modelu z wytrenowanymi już wagami (trainable=False). 

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

base_model = VGG16(weights='imagenet', include_top=False, input_shape=x_train[0].shape)
base_model.trainable = False


In [10]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(150, activation='relu')
prediction_layer = layers.Dense(43, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    prediction_layer
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 150)               76950     
                                                                 
 dense_1 (Dense)             (None, 43)                6493      
                                                                 
Total params: 14,798,131
Trainable params: 83,443
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
y_train.shape

(36288,)

In [74]:
import pandas as pd
y_train = pd.get_dummies(y_train).to_numpy()
y_validation = pd.get_dummies(y_validation).to_numpy()
y_test = pd.get_dummies(y_test).to_numpy()

In [75]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
908/908 [==============================] - 41s 40ms/step - loss: 2.0640 - accuracy: 0.6008 - val_loss: 1.0568 - val_accuracy: 0.7152
Epoch 2/50
908/908 [==============================] - 35s 39ms/step - loss: 0.7532 - accuracy: 0.7740 - val_loss: 0.7749 - val_accuracy: 0.7852
Epoch 3/50
908/908 [==============================] - 35s 39ms/step - loss: 0.5510 - accuracy: 0.8295 - val_loss: 0.7730 - val_accuracy: 0.7834
Epoch 4/50
908/908 [==============================] - 35s 39ms/step - loss: 0.4541 - accuracy: 0.8551 - val_loss: 0.6933 - val_accuracy: 0.8143
Epoch 5/50
908/908 [==============================] - 35s 39ms/step - loss: 0.3894 - accuracy: 0.8763 - val_loss: 0.6481 - val_accuracy: 0.8330
Epoch 6/50
908/908 [==============================] - 35s 39ms/step - loss: 0.3391 - accuracy: 0.8892 - val_loss: 0.6649 - val_accuracy: 0.8325
Epoch 7/50
908/908 [==============================] - 35s 39ms/step - loss: 0.3023 - accuracy: 0.9006 - val_loss: 0.6673 - val_accuracy:

In [19]:
model.predict(x_train[:2])

1/1 [==============================] - 0s 23ms/step


array([[0.0000000e+00, 3.4126278e-36, 2.4456801e-34, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.3031604e-32,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 7.4905679e-31, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 1.0549303e-37, 0.0000000e+00,
        6.7797821e-36, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        4.5613204e-38, 0.0000000e+00, 7.4406331e-34, 0.0000000e+00,
        0.0000000e+00, 7.9408963e-36, 0.0000000e+00],
       [4.1512739e-12, 6.6080820e-06, 7.8983510e-07, 1.5811435e-06,
        1.3831764e-05, 3.1484937e-08, 3.7811067e-11, 2.5975558e-10,
        3.4432971e-06, 2.7480095e-08, 1.0203946e-10, 5.3578051e-04,
        9.3616372e-06, 3.6711779e-06, 2.4849458e-08, 2.5991131

*TODO: omówienie (100 słów)*

omówienie will be here...

### Zadanie 2: Dodatkowe sieci ###
**(Zadanie na ocenę 4, po wykonaniu  zadania 1)**

Przeprowadź to samo na dwóch dodatkowych sieciach i omów wyniki (100 słów). 

Czyli jeśli w zadaniu 1 użyliśmy np. VGG to teraz wybieramy sobie np. ResNet i MobileNet. 

## Let's try with pytorch lightning

In [1]:
import h5py
filename = "dataset_ts_original.hdf5"

with h5py.File(filename, "r") as f:
    x_test = f['x_test'][()]
    x_train = f['x_train'][()]
    y_test = f['y_test'][()]
    y_train = f['y_train'][()]
    x_validation = f['x_validation'][()]
    y_validation = f['y_validation'][()]

In [2]:
import numpy as np
import torch

from torch.utils.data import TensorDataset, DataLoader


# train
tensor_x_train = torch.from_numpy(x_train).permute(0, 3, 1, 2)
tensor_y_train = torch.Tensor(y_train)
#tensor_y_train = tensor_y_train.type(torch.FloatTensor)
tensor_dataset_train = TensorDataset(tensor_x_train, tensor_y_train)
# val
tensor_x_val = torch.from_numpy(x_validation).permute(0, 3, 1, 2)
tensor_y_val = torch.Tensor(y_validation)
#tensor_y_val = tensor_y_val.type(torch.FloatTensor)

tensor_dataset_val = TensorDataset(tensor_x_val, tensor_y_val)
#test
tensor_x_test = torch.from_numpy(x_test).permute(0, 3, 1, 2)
tensor_y_test = torch.Tensor(y_test)
#tensor_y_test = tensor_y_test.type(torch.FloatTensor)

tensor_dataset_test = TensorDataset(tensor_x_test, tensor_y_test)

In [3]:
tensor_y_train.dtype

torch.float32

In [4]:
import pytorch_lightning as pl

class ResNet50Model(pl.LightningModule):
    def __init__(self, pretrained=True, in_channels = 3, num_classes = 16, lr=3e-4, freeze=False):
        super(ResNet50Model, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.lr = lr
        
        self.model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        
        if freeze:
            for param in self.model.parameters():
                param.requires_grad = False
        
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 128),
            nn.Dropout(0.3),
            nn.Linear(128, self.num_classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
        
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=self.num_classes)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=self.num_classes)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=self.num_classes)
        
        
    def forward(self, x):
        return self.model(x)
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2)
        return [optimizer], [scheduler]
    
    
    def training_step(self, batch, batch_idx):
        
        x, y = batch
        
        preds = self.model(x)

        y = y.type(torch.LongTensor)
        x, y = x.to(device), y.to(device)
        
        loss = self.loss_fn(preds, y)
        self.train_acc(torch.argmax(preds, dim=1), y)
        
        self.log('train_loss', loss.item(), on_epoch=True)
        self.log('train_acc', self.train_acc, on_epoch=True)
        
        return loss
    
    
    def validation_step(self, batch, batch_idx):
        
        x,y = batch
        
        preds = self.model(x)
        
        y = y.type(torch.LongTensor)
        x, y = x.to(device), y.to(device)
        
        loss = self.loss_fn(preds, y)
        self.val_acc(torch.argmax(preds, dim=1), y)
        
        self.log('val_loss', loss.item(), on_epoch=True)
        self.log('val_acc', self.val_acc, on_epoch=True)
        
    
    def test_step(self, batch, batch_idx):
        
        x,y = batch
        preds = self.model(x)
        self.test_acc(torch.argmax(preds, dim=1), y)
        
        self.log('test_acc', self.test_acc, on_epoch=True)

In [5]:
import torchvision.models as models
import torch.nn as nn
import torchmetrics

In [6]:
import torch
from sklearn.model_selection import StratifiedKFold
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device("cuda:0")
num_folds = 4
kf = StratifiedKFold(num_folds)
lr = 3e-4
logs = dict()

for fold, (train_fold, val_fold) in enumerate(kf.split(X=x_train, y=y_train), start=1):
    train_dataloader = torch.utils.data.DataLoader(tensor_dataset_train,batch_size=64,num_workers=8)
    val_dataloader = torch.utils.data.DataLoader(tensor_dataset_val, batch_size=64,num_workers=8)
    test_dataloader = torch.utils.data.DataLoader(tensor_dataset_test, batch_size=128, num_workers=8)
    
    model = ResNet50Model(num_classes=43)
    trainer = pl.Trainer(accelerator='gpu', 
                         max_epochs=10,
                         callbacks=[
                         EarlyStopping(monitor="val_loss", 
                                       mode="min",
                                       patience=2)])
    
    model.hparams.lr = lr
    
    print(f"\n\n\n{'=='*20} FOLD {fold} / {num_folds} {'=='*20}")
    
    trainer.fit(model, train_dataloader, val_dataloader)
    metrics = trainer.logged_metrics
    trainer.test(model, test_dataloader)
    
    logs[f'fold{fold}'] = {
        'train_loss': metrics['train_loss_epoch'].item(),
        'val_loss': metrics['val_loss'].item(),
        'train_acc': metrics['train_acc_epoch'].item(),
        'val_acc': metrics['val_acc'].item()
    }
    
    print(f"Train Loss: {logs[f'fold{fold}']['train_loss']} | Train Accuracy: {logs[f'fold{fold}']['train_acc']}")
    print(f"Val Loss: {logs[f'fold{fold}']['val_loss']} | Val Accuracy: {logs[f'fold{fold}']['val_acc']}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs





======================================== FOLD 1 / 4 ========================================


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | ResNet             | 23.8 M
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
4 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
23.8 M    Trainable params
0         Non-trainable params
23.8 M    Total params
95.103    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9993571043014526
────────────────────────────────────────────────────────────────────────────────
Train Loss: 0.0001943597017088905 | Train Accuracy: 1.0
Val Loss: 0.004916341044008732 | Val Accuracy: 0.9986334443092346


In [7]:
first_parameter = next(model.parameters())
input_shape = first_parameter.size()
input_shape

torch.Size([64, 3, 7, 7])

*TODO: omówienie (100 słów)*

### Zadanie 3: Trening od zera i porównanie ###
**(Zadanie na ocenę 5, po wykonaniu zadania 1 i 2)**

Spróbuj skonstruować swój własny model i wytrenować go 'od zera' na tych samych danych. Porównaj i omów swój ekeperyment i wyniki (100 słów).

reference: https://keras.io/api/applications/#usage-examples-for-image-classification-models

*TODO: omówienie (100 słów)*

To wszystko, dziękuję. Wypełniony notatnik zapisz jako .ipynb i oddaj w Teams. 